Face Detection test using Haarcascades

In [1]:
import cv2, queue, threading, time

# bufferless VideoCapture
class VideoCapture:

  def __init__(self, name):
    self.cap = cv2.VideoCapture(name)
    self.q = queue.Queue()
    t = threading.Thread(target=self._reader)
    t.daemon = True
    t.start()

  # read frames as soon as they are available, keeping only most recent one
  def _reader(self):
    while True:
      ret, frame = self.cap.read()
      if not ret:
        break
      if not self.q.empty():
        try:
          self.q.get_nowait()   # discard previous (unprocessed) frame
        except queue.Empty:
          pass
      self.q.put(frame)

  def read(self):
    return self.q.get()

cap = VideoCapture(1)
faceCascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 255, 0)
fontsize = 2
while True:
  frame = cap.read()
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(
    gray,     
    scaleFactor=1.2,
    minNeighbors=5,     
    minSize=(10, 10)
  )
  for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    #print(f'X : {x} \t Y : {y}')
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    position = (x, y)
    try :
        if x < 50 :
            cv2.putText(frame, f'x : {x} : LEFT', position, font, fontsize, color=color)
        elif x > 350 :
            cv2.putText(frame, f'x : {x} : RIGHT', position, font, fontsize, color=color)
        else :
            cv2.putText(frame, f'x : {x}', position, font, fontsize, color=color)
    except NameError :
        pass
  cv2.imshow("frame", frame)
  if chr(cv2.waitKey(1)&255) == 'q':
    break


In [1]:
import cv2
import time
cap = cv2.VideoCapture(1)
faceCascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 255, 0)
fontsize = 2

while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
        minSize=(10, 10)
    )
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        #print(f'X : {x} \t Y : {y}')
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        position = (x, y)
    try :
        if x < 50 :
            cv2.putText(frame, f'x : {x} : LEFT', position, font, fontsize, color=color)
        elif x > 350 :
            cv2.putText(frame, f'x : {x} : RIGHT', position, font, fontsize, color=color)
        else :
            cv2.putText(frame, f'x : {x}', position, font, fontsize, color=color)
    except NameError :
        pass
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
cap.release()
cv2.destroyAllWindows()

[ WARN:0@0.017] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@0.132] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
import urllib.request
import cv2
import numpy as np
import time
URL = "http://192.168.43.1:8080"
while True:    
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    img = cv2.imdecode(img_arr,-1)
    cv2.imshow('IPWebcam',img)
    
    if cv2.waitKey(1):
        break

Dataset farmer

In [ ]:
import cv2, sys, numpy, os
import time
haar_file = 'haarcascades/haarcascade_frontalface_default.xml'

datasets = 'datasets_VGG'  
  
sub_data = 'elf' #<-----
    
face_id = 0
(width, height) = (224, 224)

path = os.path.join(datasets, sub_data)
if not os.path.isdir(path):
    os.mkdir(path)


face_cascade = cv2.CascadeClassifier(haar_file)
webcam = cv2.VideoCapture(0) 

count = 1
while count < 300: 
    (_, im) = webcam.read()
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
        face = im[y :y + h , x :x + w]
        face_resize = cv2.resize(face, (width, height))
        cv2.imwrite('% s/% s.png' % (path, count), face_resize)
    count += 1
      
    cv2.imshow('OpenCV', im)
    key = cv2.waitKey(10)
    if key == 27:
        break
    time.sleep(0.1)
cv2.destroyAllWindows()

Load image to TensorFlow

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
images = []
images.append(tf.data.Dataset.list_files('datasets/elf/*.png'))
images.append(tf.data.Dataset.list_files('datasets/petch/*.png'))
images.append(tf.data.Dataset.list_files('datasets/bus/*.png'))

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
images[0] = images[0].map(load_image)
images[1] = images[1].map(load_image)
images[2] = images[2].map(load_image)

In [ ]:
type(images[0])

In [ ]:
image_generator_elf = images[0].batch(5).as_numpy_iterator()
plot_images_elf = image_generator_elf.next()

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, image in enumerate(plot_images_elf):
    ax[idx].imshow(image) 
plt.show()

In [ ]:
image_generator_petch = images[1].batch(5).as_numpy_iterator()
plot_images_petch = image_generator_petch.next()
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, image in enumerate(plot_images_petch):
    ax[idx].imshow(image) 
plt.show()

In [ ]:
image_generator_bus = images[2].batch(5).as_numpy_iterator()
plot_images_bus = image_generator_bus.next()
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, image in enumerate(plot_images_bus):
    ax[idx].imshow(image) 
plt.show()

In [9]:
label = 3
name = 'petch'

In [10]:
import os

file_name_list = os.listdir(f'datasets_VGG/{name}')

print(file_name_list)
id = 0
for f in file_name_list:
    print(f'{id} : {f}')
    id+=1

['67.png', '292.png', '52.png', '206.png', '87.png', '276.png', '83.png', '265.png', '49.png', '90.png', '58.png', '17.png', '161.png', '47.png', '114.png', '148.png', '173.png', '284.png', '163.png', '222.png', '39.png', '2.png', '80.png', '239.png', '146.png', '240.png', '93.png', '61.png', '289.png', '8.png', '7.png', '260.png', '102.png', '43.png', '147.png', '132.png', '118.png', '202.png', '51.png', '99.png', '4.png', '119.png', '152.png', '106.png', '219.png', '75.png', '236.png', '81.png', '207.png', '160.png', '200.png', '286.png', '9.png', '264.png', '153.png', '25.png', '76.png', '134.png', '154.png', '136.png', '166.png', '288.png', '105.png', '164.png', '60.png', '263.png', '210.png', '78.png', '196.png', '277.png', '145.png', '201.png', '195.png', '34.png', '84.png', '110.png', '44.png', '133.png', '103.png', '180.png', '155.png', '1.png', '29.png', '203.png', '18.png', '120.png', '150.png', '290.png', '281.png', '91.png', '128.png', '38.png', '77.png', '35.png', '255.png

In [11]:
import os

id = 0

for f in file_name_list:
    print(f'datasets_VGG/{name}/{f} --> dataset_LBPH2/User.{label}.{id}.png')
    os.rename(f'datasets_VGG/{name}/{f}', f'dataset_LBPH2/User.{label}.{id}.png')
    id+=1

datasets_VGG/petch/67.png --> dataset_LBPH2/User.3.0.png
datasets_VGG/petch/292.png --> dataset_LBPH2/User.3.1.png
datasets_VGG/petch/52.png --> dataset_LBPH2/User.3.2.png
datasets_VGG/petch/206.png --> dataset_LBPH2/User.3.3.png
datasets_VGG/petch/87.png --> dataset_LBPH2/User.3.4.png
datasets_VGG/petch/276.png --> dataset_LBPH2/User.3.5.png
datasets_VGG/petch/83.png --> dataset_LBPH2/User.3.6.png
datasets_VGG/petch/265.png --> dataset_LBPH2/User.3.7.png
datasets_VGG/petch/49.png --> dataset_LBPH2/User.3.8.png
datasets_VGG/petch/90.png --> dataset_LBPH2/User.3.9.png
datasets_VGG/petch/58.png --> dataset_LBPH2/User.3.10.png
datasets_VGG/petch/17.png --> dataset_LBPH2/User.3.11.png
datasets_VGG/petch/161.png --> dataset_LBPH2/User.3.12.png
datasets_VGG/petch/47.png --> dataset_LBPH2/User.3.13.png
datasets_VGG/petch/114.png --> dataset_LBPH2/User.3.14.png
datasets_VGG/petch/148.png --> dataset_LBPH2/User.3.15.png
datasets_VGG/petch/173.png --> dataset_LBPH2/User.3.16.png
datasets_VGG/pet

In [ ]:
print(file_name_list)